<a href="https://colab.research.google.com/github/vneumannufprbr/Series-Temporales/blob/main/LSTMDemandaDadosBrutos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# Nova seção

In [ ]:
# Importações
import tensorflow as tf
tf.__version__
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.optimizers import SGD, Adam
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
# incluido por VN
# correlograma e sazonalidade para determinar tamanho da janela (window size)
import statsmodels.api as sm
# Livraria para preprocessamento
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import FunctionTransformer
from scipy.stats import iqr
# Livaria para autoajuste de parâmetros
!pip install keras-tuner
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
# Livaria pandas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.core import window
from pandas.core import indexes


In [ ]:
# Carga da base
#df = pd.read_csv("/content/drive/Othercomputers/Meu computador/Google Drive/Especialização IA UFPR/AAA-Curso IA UFPR/Arquitetura de Dados/Artigos de AI aplicada a Sistemas de Energia/ENCONTRO 02/archive/energy_dataset.csv",  low_memory=False) # dtype=dtype_dict,
!git clone https://github.com/vneumannufprbr/Series-Temporales.git
df = pd.read_csv('Series-Temporales/energy_dataset.csv',  low_memory=False) # dtype=dtype_dict,
dtype_dict = {
    'time': str,
    'generation biomass': float,
    'generation fossil brown coal/lignite': float,
    'generation fossil gas': float,
    'generation fossil hard coal': float,
    'generation fossil oil': float,
    'generation hydro pumped storage consumption': float,
    'generation hydro run-of-river and poundage': float,
    'generation hydro water reservoir': float,
    'generation nuclear': float,
    'generation solar': float,
    'generation waste': float,
    'generation wind onshore': float,
    'forecast solar day ahead': float,
    'forecast wind onshore day ahead': float,
    'total load forecast': float,
    'total load actual': float,
    'price day ahead': float,
     # ... especificar outros tipos de dados das colunas aqui
    'price actual': float
}
print(df.dtypes)
df

In [ ]:
df.index = pd.to_datetime(df['time'], format='mixed', utc=True)
df.columns

In [ ]:
ger_real = df['total load actual']
ger_solar_predito_origem = df['total load forecast']
#ger_real[-240:].plot(label="Valores reais")
#plt.figure(figsize=(10, 6))
plt.plot(ger_real[-24*30:], label='Valores reais')
plt.plot(ger_solar_predito_origem[-24*30:], label='Valores preditos originais')
plt.legend()
plt.grid(True)
plt.ylabel('Potência (MW)')
plt.xlabel('Data (Ano-Mês-Dia)')
plt.xticks(rotation='vertical');

In [ ]:
# Pré-processamento
# incluido por VN

# Use o transformador para aplicar a função aos seus dados
# Defina uma função personalizada para calcular o IQR
#series=ger_real.values.reshape(-1, 1)
#def interquartile(series):
#  return iqr(series)

# Crie um transformador de função usando a função personalizada
#transformer = FunctionTransformer(interquartile, validate=False)

# Use o transformador para aplicar a função aos seus dados
#series = transformer.transform(series)
# Os outliers foram eliminados

#dados_transformados = transformer.transform(data)
# Os outliers foram eliminados

#dados_transformados.head()
#plt.plot(data)
#plt.show()
#values1 = df['generation_solar']
#series = values1.values.reshape(-1, 1)

# Calcular o IQR usando a função quantile() do pandas
Q1 = ger_real.quantile(0.25)
Q3 = ger_real.quantile(0.75)
IQR = Q3 - Q1

# Definir limites superior e inferior
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar outliers
ger_real_filt = ger_real[(ger_real >= lower_bound) & (ger_real <= upper_bound)]

# Verificar a quantidade de outliers removidos
outliers_removed = len(ger_real) - len(ger_real_filt)
print("Outliers removidos:", outliers_removed)
ger_real=ger_real_filt
series = ger_real.values.reshape(-1, 1)
series = series.astype('float64')

# Normalizar os dados da série
scaler = MinMaxScaler(feature_range=(0, 1))
series = scaler.fit_transform(series)
ger_real=pd.Series(series.flatten())

In [ ]:
# Correlograma e sazonalidade para determinar tamanho da janela (window size)
# Calcular o correlograma
acf = sm.tsa.acf(ger_real)

# Plotar o correlograma
lags = range(1, len(acf)+1)
#plt.figure(figsize=(10, 6))
plt.bar(lags, acf)
plt.xlabel('Lag (atraso)')
plt.ylabel('Autocorrelação')
#plt.title('Correlograma')
plt.grid(True)
plt.show()

In [ ]:
# Definir a série temporal para identificar a sazonalide
# Definir a série temporal
dates = pd.date_range(start='2018-01-01', periods=365, freq='d')
series2 = pd.Series(ger_real.values[-365:], index=dates)

# Converter o índice para um DatetimeIndex
series2.index = pd.DatetimeIndex(series2.index)

# Decompor a série temporal
decomposition = sm.tsa.seasonal_decompose(series2, model='additive')

# Obter as componentes
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

# Plotar as componentes
#plt.figure(figsize=(10, 8))
plt.subplot(411)
plt.grid(True)
plt.plot(series2, label='Dados originais normalizados do último ano')
plt.legend(loc='best')
plt.subplot(412)
plt.grid(True)
plt.plot(trend, label='Tendência')
plt.legend(loc='best')
plt.subplot(413)
plt.grid(True)
plt.plot(seasonal, label='Sazonalidade')
plt.legend(loc='best')
plt.subplot(414)
plt.grid(True)
plt.plot(residual, label='Resíduo')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
# Tamanho da janela que o correlograma e sazonalidade sugerem: 24h
window_size = 24
from pandas.core import window
def df_to_X_y(df, window_size):
  df_as_np = df.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [[a] for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size]
    y.append(label)
  return np.array(X), np.array(y)

In [ ]:
X, y = df_to_X_y(ger_real, window_size)
X.shape, y.shape

In [ ]:
X_train, y_train = X[:int(0.8*len(ger_real))], y[:int(0.8*len(ger_real))]
X_val, y_val  = X[int(0.8*len(ger_real)):int(0.9*len(ger_real))], y[int(0.8*len(ger_real)):int(0.9*len(ger_real))]
X_test, y_test = X[int(0.9*len(ger_real)):], y[int(0.9*len(ger_real)):]
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

In [ ]:
# Autoajuste de hiperparâmetros
# Se usa a função keras_tuner do pacote Keras-Tuner para fazer a busca
# automática de hiperparâmetros para o modelo LSTM. O Keras-Tuner é uma
# biblioteca que permite otimizar automaticamente os hiperparâmetros de um
# modelo de aprendizado de máquina.

# O build_model é uma função que constrói o modelo LSTM com os hiperparâmetros
# ajustáveis. O tuner é criado usando a função RandomSearch, que realiza uma
# busca aleatória de hiperparâmetros. É possível definir os hiperparâmetros
# a serem ajustados usando as classes do Keras-Tuner, como Int para valores
# inteiros e Choice para escolha entre opções.

# A busca de hiperparâmetros é realizada chamando o método search do tuner,
# passando os dados de treinamento e validação, bem como o número de épocas
# para treinamento. Após a busca, você pode obter o melhor modelo encontrado
# usando o método get_best_models.

# Definir a função de construção do modelo
timesteps = window_size
input_dim = 1
def build_model(hp):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32),
                            dropout=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1),
                                input_shape=(timesteps, input_dim)))
    model.add(keras.layers.Dense(units=1, activation='linear'))
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[0.001, 0.01, 0.1])),
                  loss='huber')
    return model

# Definir os hiperparâmetros a serem ajustados
hp = HyperParameters()
hp.Int('units', min_value=32, max_value=512, step=32)
hp.Choice('learning_rate', values=[0.001, 0.01, 0.1])
hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
hp.Choice('batch_size', values=[16, 32, 64, 128])  # Adicionando o hiperparâmetro batch_size

# Criar o tuner
tuner = RandomSearch(build_model, objective='val_loss', max_trials=10, hyperparameters=hp)

# Realizar a busca de hiperparâmetros
#tuner.search(x=train_data, y=train_labels, epochs=10, validation_data=(val_data, val_labels))
tuner.search(x=X_train, y=y_train, epochs=10, validation_data=(X_val, y_val))

# Obter o melhor modelo encontrado
best_model = tuner.get_best_models(num_models=1)[0]

best_model.summary()

In [ ]:
best_model.summary()

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import Huber
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, LSTM, Dense

#bath=32
#feature=1
#timesteps=24
#X_train_time_major = np.transpose(X_train, (1, 0, 2))
model1 = Sequential()
model1.add(InputLayer((window_size, 1)))
model1.add(LSTM(192, dropout=0.0))#, dropout=0.1,time_major=True, , time_major=True))
# # A camada de dropout=0.3 ajuda a prevenir overfitting
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

In [ ]:
cp1= ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=Huber(), optimizer=Adam(learning_rate=0.01), metrics =[RootMeanSquaredError()])
# função de perda huber: model1.compile(loss=MeanSquaredError(), loss=Huber()) é geralmente utilizada em problemas
# de regressão, onde se deseja uma função de perda menos sensível a outliers.
#model1.compile(loss="mse", optimizer=Adam(learning_rate=0.1), run_eagerly=True)


In [ ]:

r=model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, callbacks=[cp1], batch_size=64)

In [ ]:
#Plotar a função de perda e RMSE
#plt.figure(figsize=(10, 6))
plt.plot(r.history["loss"], label="Função de perda - LSTM")
plt.plot(r.history["val_loss"], label="Val. Função de perda da validação- LSTM")
plt.plot(r.history["root_mean_squared_error"], label="RMSE-LSTM")
plt.plot(r.history["val_root_mean_squared_error"], label="Val. RMSE-LSTM")
plt.legend()
plt.grid(True)
plt.xlabel('Época')

In [ ]:
from tensorflow.keras.models import load_model
model1 = load_model('model1/')

In [ ]:
train_predictions = model1.predict(X_train).flatten()
train_results = pd.DataFrame(data={'Train Predictions': train_predictions, 'Actuals':y_train})
#train_results

# Recuperar valores nominais originais
train_results['Train Predictions'] = scaler.inverse_transform(train_results['Train Predictions'].values.reshape(-1, 1)).flatten()
train_results['Actuals'] = scaler.inverse_transform(train_results['Actuals'].values.reshape(-1, 1)).flatten()
#train_results


In [ ]:
#plt.figure(figsize=(10, 6))
plt.plot(train_results['Train Predictions'][int(0.8*len(ger_real))-48:int(0.8*len(ger_real))], label="Predito pelo treinamento")
plt.plot(train_results['Actuals'][int(0.8*len(ger_real))-48:int(0.8*len(ger_real))], label="Valores reais")
#plt.plot(ger_solar_predito_origem[int(0.8*len(ger_real))-48:int(0.8*len(ger_real))], label='Predição original')
plt.legend()
plt.ylabel('Potência (MW)')
plt.xlabel('Instância (# hora)')
plt.grid(True)
plt.xticks(rotation='vertical');


In [ ]:
#plt.figure(figsize=(10, 6))
plt.plot(train_results['Train Predictions'][:168], label="Predito pelo treinamento")
plt.plot(train_results['Actuals'][:168], label="Valores reais")
#plt.plot(ger_solar_predito_origem.values[:168], label='Predição original')
plt.legend()
plt.ylabel('Potência (MW)')
plt.xlabel('Instância (# hora)')
plt.grid(True)
plt.xticks(rotation='vertical');

In [ ]:
val_predictions = model1.predict(X_val).flatten()
val_results = pd.DataFrame(data={'Val Predictions': val_predictions, 'Actuals':y_val})
#val_results

# Recuperar valores nominais originais
val_results['Val Predictions'] = scaler.inverse_transform(val_results['Val Predictions'].values.reshape(-1, 1)).flatten()
val_results['Actuals'] = scaler.inverse_transform(val_results['Actuals'].values.reshape(-1, 1)).flatten()
#val_results


In [ ]:
#plt.plot(val_results['Val Predictions'][:48])
#plt.plot(val_results['Actuals'][:48])
#plt.figure(figsize=(10, 6))
plt.plot(val_results['Val Predictions'][:48], label="Predito pela validação")
plt.plot(val_results['Actuals'][:48], label="Valores reais")
#plt.plot(ger_solar_predito_origem.values[:48], label='Predição original')
plt.legend()
plt.ylabel('Potência (MW)')
plt.xlabel('Instância (# hora)')
plt.grid(True)
plt.xticks(rotation='vertical');

In [ ]:
test_predictions = model1.predict(X_test).flatten()
test_results = pd.DataFrame(data={'Test Predictions': test_predictions, 'Actuals':y_test})
#test_results

# Recuperar valores nominais originais
test_results['Test Predictions'] = scaler.inverse_transform(test_results['Test Predictions'].values.reshape(-1, 1)).flatten()
test_results['Actuals'] = scaler.inverse_transform(test_results['Actuals'].values.reshape(-1, 1)).flatten()
#test_results

In [ ]:
#plt.plot(test_results['Test Predictions'][:48])
#plt.plot(test_results['Actuals'][:48])
#plt.figure(figsize=(10, 6))
plt.plot(test_results['Test Predictions'][-240:], label="Predito nos dados de teste")
plt.plot(test_results['Actuals'][-240:], label="Valores reais")
#plt.plot(ger_solar_predito_origem.values[48:], label='Predição original')
plt.legend()
plt.ylabel('Potência (MW)')
plt.xlabel('Instância (# hora)')
plt.grid(True)
plt.xticks(rotation='vertical');

In [ ]:
# Defina a data final dos dados de teste
data_final_teste = pd.to_datetime('2018-12-31')

# Crie as datas de validação e teste
#dates_val = pd.date_range(start='2018-01-01', end=data_final_teste, freq='D')
#dates_test = pd.date_range(start=data_final_teste + pd.DateOffset(days=1), end='2019-12-31', freq='D')
dates_val = pd.date_range(start='2019-01-01', end=data_final_teste, freq='h')
dates_test = pd.date_range(start=data_final_teste + pd.DateOffset(days=1), end='2019-01-31', freq='h')

from copy import deepcopy

recursive_predictions = []
#last_window = deepcopy(X_test[-1])   # Considera o último 1 ponto de dados como preditores
last_window = deepcopy(X_test[-720])  # Considera os últimos 720 pontos de dados como preditores
recursive_dates = np.concatenate([dates_val, dates_test])

for target_date in recursive_dates:
    next_prediction = model1.predict(np.array([last_window])).flatten()
    recursive_predictions.append(next_prediction[0])
    last_window = np.roll(last_window, -1)
    last_window[-1] = next_prediction[0]

# Crie um DataFrame com as previsões
pred_results = pd.DataFrame({'Date': recursive_dates, 'Predictions': recursive_predictions})
print(pred_results)

In [ ]:
#plt.figure(figsize=(10, 6))
# Recuperar valores nominais originais
pred_results['Predictions'] = scaler.inverse_transform(pred_results['Predictions'].values.reshape(-1, 1)).flatten()

plt.plot(pred_results['Date'], pred_results['Predictions'], label='Predição a partir de 02-01-2019')
plt.xlabel('Ano-Mês-Dia')
plt.xticks(rotation='vertical');
plt.ylabel('Demanda de energia (MWh)')
#plt.title('Valores previstos ao longo do tempo')
plt.legend()
plt.grid(True)
plt.show()
